In [1]:
# DS623 Team Project - Image Processing and PCA (using NumPy and Pandas only)

# Import allowed libraries
# The os module is part of Python's standard library, not an external or high-level framework and is
# used for navigating folders and handling file paths
import os
import numpy as np
# Image is used as a basic preprocessing step for file I/O and not processing or analysis
from PIL import Image

# Dataset path and categories (match your folder names exactly)
dataset_path = './catdogbird_dataset'
categories = ['cat', 'dog', 'bird']

# Function to load and preprocess images (grayscale, resize, flatten)
def load_images(folder, size=(128, 128)):
    images = []
    for filename in os.listdir(folder):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            path = os.path.join(folder, filename)
            img = Image.open(path).convert('L')  # Convert to grayscale
            img = img.resize(size)
            images.append(np.array(img).flatten())
    return np.array(images)

# Load the dataset into a dictionary
data = {}
for category in categories:
    folder_path = os.path.join(dataset_path, category)
    data[category] = load_images(folder_path)

# Check shape of loaded data
for category in categories:
    print(f"{category}: {data[category].shape}")

# Display sample images using PIL (open a few directly)
def show_samples(category, num_samples=3, size=(128, 128)):
    folder_path = os.path.join(dataset_path, category)
    files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    for i in range(min(num_samples, len(files))):
        path = os.path.join(folder_path, files[i])
        img = Image.open(path).convert('L').resize(size)
        print(f"Displaying {category} image: {files[i]}")
        img.show()

# Show 3 sample images from each category
for category in categories:
    print(f"\nSample {category} images:")
    show_samples(category)

cat: (100, 16384)
dog: (100, 16384)
bird: (100, 16384)

Sample cat images:
Displaying cat image: 1.jpg
Displaying cat image: 10.jpg
Displaying cat image: 100.jpg

Sample dog images:
Displaying dog image: 1.jpg
Displaying dog image: 10.jpg
Displaying dog image: 100.jpg

Sample bird images:
Displaying bird image: 1.jpg
Displaying bird image: 10.jpg
Displaying bird image: 100.jpg


In [2]:
# Save preprocessed image arrays as .npy files for future use
# This step avoids reprocessing every time the notebook is run

for category in categories:
    filename = f"{category}_images_128x128.npy"
    np.save(filename, data[category])
    print(f"Saved {category} images to {filename}")

Saved cat images to cat_images_128x128.npy
Saved dog images to dog_images_128x128.npy
Saved bird images to bird_images_128x128.npy


In [3]:
# Save the flattened grayscale images as .npy files for later analysis
save_path = './flattened_arrays'
os.makedirs(save_path, exist_ok=True)

for category in categories:
    file_path = os.path.join(save_path, f"{category}_flattened.npy")
    np.save(file_path, data[category])
    print(f"Saved {category} data to {file_path}")

Saved cat data to ./flattened_arrays\cat_flattened.npy
Saved dog data to ./flattened_arrays\dog_flattened.npy
Saved bird data to ./flattened_arrays\bird_flattened.npy


In [4]:
# Load each .npy file, confirm shape, compute the mean and variance across the dataset
loaded_data = {}

for category in categories:
    file_path = os.path.join(save_path, f"{category}_flattened.npy")
    images = np.load(file_path)
    loaded_data[category] = images

    # Print stats
    print(f"{category.upper()} shape: {images.shape}")
    print(f"{category.upper()} mean pixel value: {np.mean(images):.2f}")
    print(f"{category.upper()} pixel variance: {np.var(images):.2f}")
    print()

CAT shape: (100, 16384)
CAT mean pixel value: 126.29
CAT pixel variance: 4056.29

DOG shape: (100, 16384)
DOG mean pixel value: 130.27
DOG pixel variance: 3917.13

BIRD shape: (100, 16384)
BIRD mean pixel value: 119.83
BIRD pixel variance: 3709.12



In [5]:
# Compute and save mean image for each category
mean_path = './mean_images'
os.makedirs(mean_path, exist_ok=True)

mean_images = {}

for category in categories:
    images = loaded_data[category]
    mean_img = np.mean(images, axis=0)
    mean_images[category] = mean_img

    # Save to disk for later use in centering and PCA
    mean_file = os.path.join(mean_path, f"{category}_mean.npy")
    np.save(mean_file, mean_img)

    print(f"Saved mean image for {category} to {mean_file}")

Saved mean image for cat to ./mean_images\cat_mean.npy
Saved mean image for dog to ./mean_images\dog_mean.npy
Saved mean image for bird to ./mean_images\bird_mean.npy


In [6]:
# Load and center data by subtracting the mean image in preparation for PCA
centered_data = {}

for category in categories:
    images = loaded_data[category]

    # Load mean image from file
    mean_file = os.path.join(mean_path, f"{category}_mean.npy")
    mean_img = np.load(mean_file)

    # Subtract the mean from each image
    centered = images - mean_img
    centered_data[category] = centered

    print(f"{category.upper()} centered shape: {centered.shape}")

CAT centered shape: (100, 16384)
DOG centered shape: (100, 16384)
BIRD centered shape: (100, 16384)


In [7]:
# Compute covariance matrix for each centered category
# Each covariance matrix will be of shape (num_pixels, num_pixels)
# Since each image is 128x128, that means 128*128 = 16,384 pixels per image

cov_matrices = {}

for category in categories:
    # Get the centered data matrix (100 x 16,384)
    X = centered_data[category]

    # Compute covariance matrix (pixels are treated as features/columns)
    cov_matrix = np.cov(X, rowvar=False)

    # Store for later eigen decomposition
    cov_matrices[category] = cov_matrix

    # Print confirmation
    print(f"{category.upper()} covariance matrix shape: {cov_matrix.shape}")

CAT covariance matrix shape: (16384, 16384)
DOG covariance matrix shape: (16384, 16384)
BIRD covariance matrix shape: (16384, 16384)


In [8]:
# Perform eigen decomposition (PCA) on each category's covariance matrix
# This provides eigenvalues (variance magnitude) and eigenvectors (principal directions)

eigen_pairs = {}

for category in categories:
    cov_matrix = cov_matrices[category]

    # Compute eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

    # Sort in descending order of eigenvalue magnitude
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sorted_indices]
    eigenvectors = eigenvectors[:, sorted_indices]

    # Store for later use (e.g., projection, reconstruction)
    eigen_pairs[category] = (eigenvalues, eigenvectors)

    # Print confirmation and leading eigenvalue
    print(f"{category.upper()} top eigenvalue: {eigenvalues[0]:.2f}, total components: {len(eigenvalues)}")

CAT top eigenvalue: 15826699.37, total components: 16384
DOG top eigenvalue: 10852990.81, total components: 16384
BIRD top eigenvalue: 27777403.95, total components: 16384


In [9]:
# Save eigenvalues and eigenvectors for each category so they can be loaded later without repeating
# the long PCA computation

eigen_path = './eigen_data'
os.makedirs(eigen_path, exist_ok=True)

for category in categories:
    eigenvalues, eigenvectors = eigen_pairs[category]

    eigval_file = os.path.join(eigen_path, f"{category}_eigenvalues.npy")
    eigvec_file = os.path.join(eigen_path, f"{category}_eigenvectors.npy")

    np.save(eigval_file, eigenvalues)
    np.save(eigvec_file, eigenvectors)

    print(f"Saved PCA data for {category.upper()} to {eigval_file} and {eigvec_file}")

Saved PCA data for CAT to ./eigen_data\cat_eigenvalues.npy and ./eigen_data\cat_eigenvectors.npy
Saved PCA data for DOG to ./eigen_data\dog_eigenvalues.npy and ./eigen_data\dog_eigenvectors.npy
Saved PCA data for BIRD to ./eigen_data\bird_eigenvalues.npy and ./eigen_data\bird_eigenvectors.npy


In [10]:
# Load saved eigenvalues and eigenvectors (PCA components) and confirm that the arrays look as
# expected before we proceed to dimensionality and projection

loaded_eigen = {}

for category in categories:
    eigval_file = os.path.join(eigen_path, f"{category}_eigenvalues.npy")
    eigvec_file = os.path.join(eigen_path, f"{category}_eigenvectors.npy")

    eigenvalues = np.load(eigval_file)
    eigenvectors = np.load(eigvec_file)

    loaded_eigen[category] = (eigenvalues, eigenvectors)

    # Print basic info
    print(f"{category.upper()} eigenvalues shape: {eigenvalues.shape}")
    print(f"{category.upper()} eigenvectors shape: {eigenvectors.shape}")
    print(f"{category.upper()} top eigenvalue: {eigenvalues[0]:.2f}")
    print()

CAT eigenvalues shape: (16384,)
CAT eigenvectors shape: (16384, 16384)
CAT top eigenvalue: 15826699.37

DOG eigenvalues shape: (16384,)
DOG eigenvectors shape: (16384, 16384)
DOG top eigenvalue: 10852990.81

BIRD eigenvalues shape: (16384,)
BIRD eigenvectors shape: (16384, 16384)
BIRD top eigenvalue: 27777403.95



In [11]:
# Dimensionality reduction to project each centered image onto the top k eigenvectors, reducing
# from 16,384 dimensions down to just 10 per image

# Set number of principal components
k = 10

# Prepare dictionary for projected data
projected_data = {}

for category in categories:
    X = centered_data[category]  # shape: (100, 16384)
    eigenvectors = loaded_eigen[category][1]  # shape: (16384, 16384)

    top_k_vectors = eigenvectors[:, :k]  # shape: (16384, 10)
    projections = X @ top_k_vectors      # shape: (100, 10)

    projected_data[category] = projections

    # Confirm result
    print(f"{category.upper()} projected shape: {projections.shape}")

CAT projected shape: (100, 10)
DOG projected shape: (100, 10)
BIRD projected shape: (100, 10)


In [12]:
# Save 10D projected feature vectors for each category
proj_path = './projected_data'
os.makedirs(proj_path, exist_ok=True)

for category in categories:
    proj_file = os.path.join(proj_path, f"{category}_projected.npy")
    np.save(proj_file, projected_data[category])
    print(f"Saved 10D projected data for {category.upper()} to {proj_file}")

Saved 10D projected data for CAT to ./projected_data\cat_projected.npy
Saved 10D projected data for DOG to ./projected_data\dog_projected.npy
Saved 10D projected data for BIRD to ./projected_data\bird_projected.npy


In [13]:
# Reverse PCA projection process to approximate the original images by loading the 10D projections,
# eigenvectors, and mean image

# Multiply the projection by the transpose of the top-10 eigenvectors, add the mean image back in
# to obtain the reconstructed image, then save a small sample of reconstructions for each category

# Directory to save reconstructed images (as .npy arrays)
recon_path = './reconstructed_images'
os.makedirs(recon_path, exist_ok=True)

for category in categories:
    # Load necessary components
    proj_file = os.path.join(proj_path, f"{category}_projected.npy")
    eigvec_file = os.path.join(eigen_path, f"{category}_eigenvectors.npy")
    mean_file = os.path.join(mean_path, f"{category}_mean.npy")

    projections = np.load(proj_file)
    eigenvectors = np.load(eigvec_file)
    mean_img = np.load(mean_file)

    # Select top-k eigenvectors and reconstruct images
    top_k_vectors = eigenvectors[:, :k]
    reconstructed = projections @ top_k_vectors.T + mean_img

    # Save reconstructed array
    save_file = os.path.join(recon_path, f"{category}_reconstructed.npy")
    np.save(save_file, reconstructed)

    print(f"Saved reconstructed images for {category.upper()} to {save_file}")

Saved reconstructed images for CAT to ./reconstructed_images\cat_reconstructed.npy
Saved reconstructed images for DOG to ./reconstructed_images\dog_reconstructed.npy
Saved reconstructed images for BIRD to ./reconstructed_images\bird_reconstructed.npy


In [14]:
# Evaluat reconstruction quality by loading original and reconstructed data, computing Mean Squared
# Error (MSE) across all samples for each category and report the results

# Compute MSE for each category manually using NumPy
print("\nReconstruction Error (Mean Squared Error):")

for category in categories:
    # Load original and reconstructed data
    original = loaded_data[category]
    recon_file = os.path.join(recon_path, f"{category}_reconstructed.npy")
    reconstructed = np.load(recon_file)

    # Compute MSE manually: average of squared differences
    mse = np.mean((original - reconstructed) ** 2)

    print(f"{category.upper()} reconstruction MSE: {mse:.2f}")


Reconstruction Error (Mean Squared Error):
CAT reconstruction MSE: 1434.88
DOG reconstruction MSE: 1574.67
BIRD reconstruction MSE: 1111.52


---
## REVISED MODEL FORMULATION

In [15]:
# Revised Model - Load and flatten 64×64 grayscale images
import os
import numpy as np
from PIL import Image

dataset_path = './catdogbird_dataset'
categories = ['cat', 'dog', 'bird']
revised_save_path = './flattened_arrays_64'
os.makedirs(revised_save_path, exist_ok=True)

def load_images_64(folder, size=(64, 64)):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            path = os.path.join(folder, filename)
            img = Image.open(path).convert('L')  # Convert to grayscale
            img = img.resize(size)
            images.append(np.array(img).flatten())
    return np.array(images)

revised_data = {}
for category in categories:
    folder_path = os.path.join(dataset_path, category)
    images = load_images_64(folder_path)
    revised_data[category] = images

    file_path = os.path.join(revised_save_path, f"{category}_flattened.npy")
    np.save(file_path, images)
    print(f"Saved 64x64 flattened {category} data to {file_path}")

Saved 64x64 flattened cat data to ./flattened_arrays_64\cat_flattened.npy
Saved 64x64 flattened dog data to ./flattened_arrays_64\dog_flattened.npy
Saved 64x64 flattened bird data to ./flattened_arrays_64\bird_flattened.npy


In [16]:
# Revised Model - Compute and save mean image for each 64×64 category
mean_path_64 = './mean_images_64'
os.makedirs(mean_path_64, exist_ok=True)

mean_images_64 = {}

for category in categories:
    images = revised_data[category]
    mean_img = np.mean(images, axis=0)
    mean_images_64[category] = mean_img

    mean_file = os.path.join(mean_path_64, f"{category}_mean.npy")
    np.save(mean_file, mean_img)

    print(f"Saved 64x64 mean image for {category.upper()} to {mean_file}")

Saved 64x64 mean image for CAT to ./mean_images_64\cat_mean.npy
Saved 64x64 mean image for DOG to ./mean_images_64\dog_mean.npy
Saved 64x64 mean image for BIRD to ./mean_images_64\bird_mean.npy


In [17]:
# Revised Model - Center 64×64 data by subtracting the mean image for each category
centered_data_64 = {}

for category in categories:
    images = revised_data[category]

    mean_file = os.path.join(mean_path_64, f"{category}_mean.npy")
    mean_img = np.load(mean_file)

    centered = images - mean_img
    centered_data_64[category] = centered

    print(f"{category.upper()} 64×64 centered shape: {centered.shape}")

CAT 64×64 centered shape: (100, 4096)
DOG 64×64 centered shape: (100, 4096)
BIRD 64×64 centered shape: (100, 4096)


In [18]:
# Revised Model - Compute covariance matrices for 64×64 centered data
cov_matrices_64 = {}

for category in categories:
    X = centered_data_64[category]  # shape: (100, 4096)
    cov_matrix = np.cov(X, rowvar=False)  # pixels as features (columns)
    cov_matrices_64[category] = cov_matrix

    print(f"{category.upper()} 64×64 covariance matrix shape: {cov_matrix.shape}")

CAT 64×64 covariance matrix shape: (4096, 4096)
DOG 64×64 covariance matrix shape: (4096, 4096)
BIRD 64×64 covariance matrix shape: (4096, 4096)


In [19]:
# Revised Model - Perform eigen decomposition for each 64×64 covariance matrix
eigen_pairs_64 = {}

for category in categories:
    cov_matrix = cov_matrices_64[category]

    # Compute eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

    # Sort in descending order of eigenvalue magnitude
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sorted_indices]
    eigenvectors = eigenvectors[:, sorted_indices]

    # Store the results
    eigen_pairs_64[category] = (eigenvalues, eigenvectors)

    print(f"{category.upper()} 64×64 top eigenvalue: {eigenvalues[0]:.2f}, total components: {len(eigenvalues)}")

CAT 64×64 top eigenvalue: 3952987.17, total components: 4096
DOG 64×64 top eigenvalue: 2707939.55, total components: 4096
BIRD 64×64 top eigenvalue: 6938237.19, total components: 4096


In [20]:
# Revised Model - Project 64×64 data onto top 10 principal components
k = 10  # Number of components
projected_data_64 = {}

for category in categories:
    X = centered_data_64[category]                      # Shape: (100, 4096)
    eigenvectors = eigen_pairs_64[category][1]          # Shape: (4096, 4096)

    top_k_vectors = eigenvectors[:, :k]                 # Shape: (4096, 10)
    projections = X @ top_k_vectors                     # Shape: (100, 10)

    projected_data_64[category] = projections

    print(f"{category.upper()} 64×64 projected shape: {projections.shape}")

CAT 64×64 projected shape: (100, 10)
DOG 64×64 projected shape: (100, 10)
BIRD 64×64 projected shape: (100, 10)


In [21]:
# Revised Model - Save 10D projections for each category
proj_path_64 = './projected_data_64'
os.makedirs(proj_path_64, exist_ok=True)

for category in categories:
    proj_file = os.path.join(proj_path_64, f"{category}_projected.npy")
    np.save(proj_file, projected_data_64[category])
    print(f"Saved 64x64 projected data for {category.upper()} to {proj_file}")

Saved 64x64 projected data for CAT to ./projected_data_64\cat_projected.npy
Saved 64x64 projected data for DOG to ./projected_data_64\dog_projected.npy
Saved 64x64 projected data for BIRD to ./projected_data_64\bird_projected.npy


In [22]:
# Revised Model - Save eigenvalues and eigenvectors for 64×64 data
eigen_path_64 = './eigen_data_64'
os.makedirs(eigen_path_64, exist_ok=True)

for category in categories:
    eigenvalues, eigenvectors = eigen_pairs_64[category]

    eigval_file = os.path.join(eigen_path_64, f"{category}_eigenvalues.npy")
    eigvec_file = os.path.join(eigen_path_64, f"{category}_eigenvectors.npy")

    np.save(eigval_file, eigenvalues)
    np.save(eigvec_file, eigenvectors)

    print(f"Saved 64x64 PCA data for {category.upper()} to {eigval_file} and {eigvec_file}")

Saved 64x64 PCA data for CAT to ./eigen_data_64\cat_eigenvalues.npy and ./eigen_data_64\cat_eigenvectors.npy
Saved 64x64 PCA data for DOG to ./eigen_data_64\dog_eigenvalues.npy and ./eigen_data_64\dog_eigenvectors.npy
Saved 64x64 PCA data for BIRD to ./eigen_data_64\bird_eigenvalues.npy and ./eigen_data_64\bird_eigenvectors.npy


In [23]:
# Revised Model - Reconstruct images from 10D projections
recon_path_64 = './reconstructed_images_64'
os.makedirs(recon_path_64, exist_ok=True)

for category in categories:
    # Load projections, eigenvectors, and 64×64 mean image
    proj_file = os.path.join('./projected_data_64', f"{category}_projected.npy")
    eigvec_file = os.path.join('./eigen_data_64', f"{category}_eigenvectors.npy")
    mean_file = os.path.join('./mean_images_64', f"{category}_mean.npy")

    projections = np.load(proj_file)
    eigenvectors = np.load(eigvec_file)
    mean_img = np.load(mean_file)

    # Reconstruct using top-k eigenvectors
    top_k_vectors = eigenvectors[:, :k]
    reconstructed = projections @ top_k_vectors.T + mean_img

    # Save reconstructed array
    save_file = os.path.join(recon_path_64, f"{category}_reconstructed.npy")
    np.save(save_file, reconstructed)

    print(f"Saved 64x64 reconstructed images for {category.upper()} to {save_file}")

Saved 64x64 reconstructed images for CAT to ./reconstructed_images_64\cat_reconstructed.npy
Saved 64x64 reconstructed images for DOG to ./reconstructed_images_64\dog_reconstructed.npy
Saved 64x64 reconstructed images for BIRD to ./reconstructed_images_64\bird_reconstructed.npy


In [24]:
# Revised Model - Compute MSE for 64x64 reconstructions
print("\nReconstruction Error (Mean Squared Error) for 64x64 images:")

for category in categories:
    original = revised_data[category]
    recon_file = os.path.join(recon_path_64, f"{category}_reconstructed.npy")
    reconstructed = np.load(recon_file)

    # MSE: mean of squared pixel-wise differences
    mse = np.mean((original - reconstructed) ** 2)

    print(f"{category.upper()} 64x64 reconstruction MSE: {mse:.2f}")


Reconstruction Error (Mean Squared Error) for 64x64 images:
CAT 64x64 reconstruction MSE: 1333.99
DOG 64x64 reconstruction MSE: 1440.78
BIRD 64x64 reconstruction MSE: 981.27
